<a href="https://colab.research.google.com/gist/avidale/4de1454bf41822dc862fddbd779d4cc6/finetune_rut5-base-multitask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece datasets natasha

We finetune our Russian-English T5 model on several tasks:
* Translation with https://huggingface.co/datasets/opus_wikipedia
* Paraphrasing with https://huggingface.co/datasets/tapaco
* Filling the gaps in a text
* Restoring the text from a noisy bag of words

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import torch
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/gd')

Mounted at /gd


In [ ]:
raw_model = 'cointegrated/rut5-base'  
MODEL_NAME = '/gd/MyDrive/models/rut5-base-partial'

if os.path.exists(MODEL_NAME):  # continue fine-tuning
    raw_model = MODEL_NAME
model = T5ForConditionalGeneration.from_pretrained(raw_model)
tokenizer = T5Tokenizer.from_pretrained(raw_model)

### Create the tasks

``` 
def task():
    return input_text, output_text
```

In [ ]:
opus_wiki = load_dataset("opus_wikipedia", lang1="en", lang2="ru")
len(opus_wiki)

Using custom data configuration en-ru-lang1=en,lang2=ru


Dataset opus_wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/opus_wikipedia/en-ru-lang1=en,lang2=ru/0.0.0/4a18b1be119afcbc678dac8b8f58888a10016b2ba19ea2ca0adfb4777f0d2b6b. Subsequent calls will reuse this data.


1

In [ ]:
print(len(opus_wiki['train']))

572717


In [ ]:
import random
random.choice(opus_wiki['train'])

{'id': '255231',
 'translation': {'en': 'The air-delivery system provides 50% more fresh air than is required by New York City Building Code, and a number of recycling chutes serve the entire building.',
  'ru': 'Система подачи воздуха предоставляет на 50 % больше свежего воздуха, чем требуется строительным кодексом Нью-Йорка, а несколько мусоросбросов обслуживают все здание.'}}

In [ ]:
def translate_task():
    item = random.choice(opus_wiki['train'])['translation']
    if random.random() < 0.5:
        return f'translate ru-en | {item["ru"]}', item["en"]
    else:
        return f'translate en-ru | {item["en"]}', item["ru"]

translate_task()

('translate ru-en | World Service не получает финансирование для радиопередач к Великобритании, и надежный средний прием волны был возможен на только на юго-востоке Англии на 648 kHz — и оно прекратилось в 2011 году, из-за снижения расходов.',
 '===UK===The BBC World Service does not receive funding for broadcasts to the UK, and reliable medium wave reception was possible in only southeast of England from the 648 kHz service which ceased in 2011 as a cost-cutting measure.')

In [ ]:
# mlsum = load_dataset("mlsum", 'ru')
# print(len(mlsum['train']))  # 25K

In [ ]:
# random.choice(mlsum['train'])

In [ ]:
# pd.Series([len(tokenizer.tokenize(random.choice(mlsum['train'])['text'])) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])
# 50% - 800, 75% 2K, 95% - 5K

Just ignore this task, texts are too long

In [ ]:
tapaco  = load_dataset('tapaco', 'ru')

Dataset tapaco downloaded and prepared to /root/.cache/huggingface/datasets/tapaco/ru/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698. Subsequent calls will reuse this data.


In [ ]:
tapaco

DatasetDict({
    train: Dataset({
        features: ['language', 'lists', 'paraphrase', 'paraphrase_set_id', 'sentence_id', 'tags'],
        num_rows: 251263
    })
})

In [ ]:
from collections import Counter, defaultdict
from tqdm.auto import tqdm, trange
cnt = Counter()
p2s = defaultdict(list)
for i, e in enumerate(tqdm(tapaco['train'])):
    cnt[e['paraphrase_set_id']] += 1
    p2s[e['paraphrase_set_id']].append(i)

In [ ]:
ph = random.choice(tapaco['train'])['paraphrase_set_id']

In [ ]:
def paraphrase_task():
    ph = random.choice(tapaco['train'])['paraphrase_set_id']
    texts = tapaco['train'][p2s[ph]]['paraphrase']
    random.shuffle(texts)
    return f'paraphrase | {texts[0]}', texts[1]

paraphrase_task()

('paraphrase | Том крайне наивен.', 'Том крайне доверчив.')

Use a large Russian corpus - take a file from https://wortschatz.uni-leipzig.de/en/download/Russian

In [ ]:
!wget http://pcai056.informatik.uni-leipzig.de/downloads/corpora/rus-ru_web-public_2019_1M.tar.gz

--2021-06-10 21:01:08--  http://pcai056.informatik.uni-leipzig.de/downloads/corpora/rus-ru_web-public_2019_1M.tar.gz
Resolving pcai056.informatik.uni-leipzig.de (pcai056.informatik.uni-leipzig.de)... 139.18.2.216
Connecting to pcai056.informatik.uni-leipzig.de (pcai056.informatik.uni-leipzig.de)|139.18.2.216|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206133725 (197M) [application/x-gzip]
Saving to: ‘rus-ru_web-public_2019_1M.tar.gz’

rus-ru_web-public_2 100%[===================>] 196.58M  68.6MB/s    in 2.9s    

2021-06-10 21:01:11 (68.6 MB/s) - ‘rus-ru_web-public_2019_1M.tar.gz’ saved [206133725/206133725]



In [ ]:
!tar -xsvf rus-ru_web-public_2019_1M.tar.gz

rus-ru_web-public_2019_1M/
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-inv_so.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-import.sql
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-co_s.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-sentences.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-sources.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-co_n.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-words.txt
rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-inv_w.txt


In [ ]:
import pandas as pd
import csv
fname = 'rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-sentences.txt'
df_leipzig = pd.read_csv(fname, sep='\t', header=None, quoting=csv.QUOTE_NONE)
df_leipzig.columns = ['idx', 'text']
df_leipzig.sample(3)

,idx,text
361939,361940,Как же сразу не заметна пасмурная погода и нак...
96707,96708,"Владимир Абрамов: «Великих тренеров – Бышовца,..."
929674,929675,"Церковь нажимала всё сильней, а местные всё не..."


In [ ]:
def fill_gap_task():
    text = random.choice(df_leipzig.text)  #random.choice(tapaco['train'])['paraphrase']
    words = text.split()
    if len(words) < 3:
        return fill_gap_task()
    right_id = random.randint(1, len(words)-2)
    left_id = random.randint(1, right_id)
    if random.random() < 0.5:
        filler = ['___']
    else:
        filler = [f'_{right_id-left_id+1}_']
    lhs = ' '.join(['fill |'] +  words[:left_id] + filler + words[right_id+1:])
    rhs = ' '.join(words[left_id:(right_id+1)])
    return lhs, rhs

fill_gap_task()

('fill | Yerli: ___ на полигональном уровне, но некоторые формы пиксельных шейдеров, включая текстурирование останутся.',
 'В общем-то все детали поверхностей рассчитываются')

#### Simplification

The WikiLarge corpus, translated to Russian (from https://github.com/dialogue-evaluation/RuSimpleSentEval), filetered by lenght and ngram similarity.

In [ ]:
import pandas as pd

In [ ]:
simple_filtered = pd.read_csv('/gd/MyDrive/datasets/wiki_simple_ru_filtered.tsv', sep='\t')

In [ ]:
def simplify_task(en=0.3):
    row = simple_filtered.sample(1).iloc[0]
    if en is True or isinstance(en, float) and random.random() < en:
        x, y = row.src, row.dst
    else:
        x, y = row.target_x, row.target_y

    return f'simplify | {x}', y

simplify_task()

('simplify | Уидон, Светлячок: полная серия: комментарий к "Train Job", дорожка 10 Он имеет тенденцию вести себя как "ламокс", который думает, что он самый умный человек в космосе, но время от времени сквозь этот фасад просматриваются намеки на разум, создается впечатление, что он действует глупее, чем он есть.',
 'Уидон, Светлячок: вся серия: комментарий к «Train Job», трек 10. Он думает, что он самый умный парень в космосе, но он прямо противоположный.')

### Summarization

https://github.com/IlyaGusev/gazeta

In [ ]:
# ! wget https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.jsonl

In [ ]:
#import json
#gazeta = []
#with open('gazeta_train.jsonl', 'r') as f:
#    for line in tqdm(f.readlines()):
#        gazeta.append(json.loads(line))

In [ ]:
#random.choice(gazeta)

In [ ]:
# pd.Series([len(random.choice(gazeta)['text']) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

In [ ]:
# pd.Series([len(tokenizer.tokenize(random.choice(gazeta)['text'])) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

In [ ]:
#gazeta_short = [g for g in gazeta if len(g['text']) <= 5000]
#print(len(gazeta))  # 52400
#print(len(gazeta_short))  # 37980

In [ ]:
#pd.Series([len(tokenizer.tokenize(random.choice(gazeta_short)['text'])) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

In [ ]:
#def sumarize_task():
#    row = random.choice(gazeta_short)
#    return f'summarize | {row["text"]}', row["summary"]
#
#sumarize_task()

Training data for https://huggingface.co/IlyaGusev/rubert_telegram_headlines

In [ ]:
!wget https://www.dropbox.com/s/ykqk49a8avlmnaf/ru_all_split.tar.gz

--2021-06-10 21:01:22--  https://www.dropbox.com/s/ykqk49a8avlmnaf/ru_all_split.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ykqk49a8avlmnaf/ru_all_split.tar.gz [following]
--2021-06-10 21:01:22--  https://www.dropbox.com/s/raw/ykqk49a8avlmnaf/ru_all_split.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc314301305ae7964b78b0a38c5d.dl.dropboxusercontent.com/cd/0/inline/BQKrAXoOeirTxLMhZUDFXB_b1sX76cFkcHWwHVC9WYfLsCuD-f1CVFlKQvaCHhEhJvdX2NYPNQfqXZf_6pu2cyx9XW-qgQ-53lpcoyLrXxZ1rNWHSoNCMtP5y5JV2CETR9TYnUPC-TgQBsekqUh_OUY9/file# [following]
--2021-06-10 21:01:22--  https://uc314301305ae7964b78b0a38c5d.dl.dropboxusercontent.com/cd/0/inline/BQKrAXoOeirTxLMhZUDFXB_b1sX76cFkcHWwHVC9WYfLsCuD-f1CVFlKQvaCHhEhJv

In [ ]:
!tar -xvzf ru_all_split.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
ru_all_train.jsonl
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
ru_all_val.jsonl
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
ru_all_test.jsonl


In [ ]:
import json
tg = []
with open('ru_all_train.jsonl', 'r') as f:
    for line in tqdm(f.readlines()):
        tg.append(json.loads(line))

In [ ]:
random.choice(tg)

{'text': 'Около 400 очагов новой коронавирусной инфекции возникло в больницах РФ, сообщили в  Министерстве здравоохранения России . Как рассказал на заседании Госдумы глава ведомства  Михаил Мурашко , россияне небезосновательно опасаются посещать медучреждения. "Пациенты опасаются — в общем, обоснованно опасаются — в этот период идти на большое вмешательство, я уже говорил о том, что 400 очагов возникло в больницах, заносы, несмотря на все профилактические меры, существуют", — сказал Мурашко (цитата по  ТАСС ). Министр также напомнил, что из-за пандемии коронавируса объёмы оказания плановой медицинской помощи снизились во всём мире. Ранее "ДП" писал о том, что в Петербурге, по последним данным, коронавирусом  заразились  почти 1,5 тыс. медработников. При этом общее число заболевших в городе 13 мая  приблизилось  к отметке в 8,5 тыс. человек. Выделите фрагмент с текстом ошибки и нажмите Ctrl+Enter',
 'timestamp': 1589380140,
 'title': 'Минздрав сообщил о 400 очагах коронавируса в больни

In [ ]:
pd.Series([len(random.choice(tg)['text']) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

0.25      731.75
0.50     1053.00
0.75     1582.00
0.90     2543.20
0.95     3631.20
0.99     7760.96
1.00    13118.00
dtype: float64

In [ ]:
pd.Series([len(tokenizer.tokenize(random.choice(tg)['text'])) for _ in range(1000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

0.25     208.00
0.50     304.50
0.75     474.00
0.90     731.50
0.95     959.15
0.99    2306.97
1.00    9433.00
dtype: float64

In [ ]:
def headline_task():
    row = random.choice(tg)
    if len(row["text"]) > 3000:
        return headline_task()
    return f'headline | {row["text"]}', row["title"]

headline_task()

('headline | В квартальном отчете "Газпрома" отметили, что НАК "Нафтогаз Украины" подала ходатайство в суд Латвии о приведении в исполнение решения Стокгольмского арбитража по делу против российского монополиста, передает " Укринформ ". "5 ноября 2019 года ПАО "Газпром" стало известно, что НАК "Нафтогаз Украины" подала ходатайство в суд Видземского пригорода г. Рига (Латвия) о признании и приведении в исполнение на территории Латвии решения Стокгольмского арбитража по транзитному спору от 28 февраля 2018 года, а также о принятии обеспечительных мер", – отметили в сообщении. В "Газпроме" подчеркнули, что заседание суда по этому ходатайству запланировано на конец апреля 2020 года. "Газпром" изучает возможности по защите своих интересов',
 '"Нафтогаз" подал ходатайство об аресте активов "Газпрома" в Латвии – подробности')

## Conversatons and answers

Dialogues from fiction, collected in https://github.com/Koziev/NLP_Datasets

In [ ]:
!wget https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Conversations/Data/ru.conversations.txt

--2021-06-10 21:02:27--  https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Conversations/Data/ru.conversations.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9718314 (9.3M) [text/plain]
Saving to: ‘ru.conversations.txt’

ru.conversations.tx 100%[===================>]   9.27M  49.3MB/s    in 0.2s    

2021-06-10 21:02:28 (49.3 MB/s) - ‘ru.conversations.txt’ saved [9718314/9718314]



In [ ]:
with open('ru.conversations.txt', 'r') as f:
    blobs = f.read().split('\n\n')
print(len(blobs))

84921


In [ ]:
def reply_task():
    b = random.choice(blobs)
    phrases = b[2:].split('\n- ')
    if len(phrases) < 2:
        return reply_task()
    split_point = random.randint(1, len(phrases)-1)
    prefix = '\n\n'.join(phrases[:split_point])
    return f'reply | {prefix}', phrases[split_point]

reply_task()

('reply | Давно бы так!', 'Товарищ, вы будете сейчас выходить?')

Samples from otvet.mail.ru

In [ ]:
import pandas as pd
mailru_df = pd.DataFrame(pd.read_pickle('/gd/MyDrive/datasets/nlp/mailru.random100k_from_first_1500k.pkl'))

In [ ]:
def answer_task():
    row = mailru_df.sample(1).iloc[0]
    return f'answer | {row.q}', row.a

answer_task()

('answer | Она ангел', 'Все мы тут ангелы :)')

quizes

In [ ]:
import pandas as pd
quiz_df = pd.DataFrame(pd.read_csv('/gd/MyDrive/datasets/nlp/quiz.tsv', sep='\t'))

In [ ]:
def quiz_task():
    row = quiz_df.sample(1).iloc[0]
    return f'quiz | {row.q}', row.a

quiz_task()

('quiz | Задорнизмы. “Знаете ли вы, что чиновники очень любят смотреть на морской прибой с берега: [...три слова пропущено...]!”',
 'откат за откатом')

### A special Russian text normalization task

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


def text2doc(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

In [ ]:
from pymorphy2 import MorphAnalyzer
anl = MorphAnalyzer()

from natasha.morph.vocab import OC_UD_INDEX, OC_UD_FEATS, OC_UD_POS 
from natasha.norm import normal_pos
fvalue_pm_dict = {}
nat2pm = {}

for fname, fvalue_pm, fvalue_nat in OC_UD_FEATS:
    fvalue_pm_dict[fvalue_pm.lower()] = (fname, fvalue_nat)
    nat2pm[(fname, fvalue_nat)] = fvalue_pm.lower()

In [ ]:
from collections import Counter, defaultdict

feature_counter = defaultdict(Counter)
pos_counter = Counter()

IMPORTANT_POS = {'NOUN', 'PROPN', 'VERB', 'ADJ', 'DET', 'PRON', 'PART', 'ADV', 'NUM', 'X', 'INTJ'}
HELPER_POS = {'ADP', 'PROPN', 'PUNCT', 'SCONJ', 'CCONJ', 'AUX', 'SYM'}

for i, item in enumerate(tqdm(tapaco['train'])):
    if i > 10000:
        break
    doc = text2doc(item['paraphrase'])
    for token in doc.tokens:
        for k, v in token.feats.items():
            feature_counter[k][v] += 1
        pos_counter[token.pos] += 1
        
        if token.pos not in IMPORTANT_POS.union(HELPER_POS):
            print(token.text)
            print(token.pos)
            print()

In [ ]:
def find_best_parse(token, parses):
    feats = {(k, v) for k, v in token.feats.items()}
    #print(feats)
    scores = []
    for p in parses:
        cand_feats = set()
        score = 1
        for g in p.tag.grammemes:
            if g in OC_UD_POS:
                if str(g) == normal_pos(token.pos) or OC_UD_POS[g] == normal_pos(token.pos):
                    score -= 1
            elif g in fvalue_pm_dict:
                cand_feats.add(fvalue_pm_dict[g])
            elif g in {'Sgtm', 'Geox', 'Name', 'intr', 'tran', 'intg', 'UNKN'}:
                continue
        score = len(feats.difference(cand_feats)) + len(cand_feats.difference(feats)) - p.score
        scores.append(score)
        #print(cand_feats, score)
    return parses[np.argmin(scores)]

In [ ]:
import random

def reinflect(token, p_lemma=0.5) -> str:
    """ Try to reinflect a token into something random """
    if token.pos == 'PUNCT':
        return token.text
    if token.lemma and random.random() < p_lemma:
        return token.lemma
    parses = anl.parse(token.text)
    bp = find_best_parse(token, parses)
    if not bp:
        return token.lemma or token.text
    new_feats = []
    for k, v in token.feats.items():
        if k in {'Animacy', 'Aspect'}:
            continue
        if bp.tag.POS in {'NOUN'} and k in {'Gender'}:
            continue
        if 'Sgtm' in bp.tag and k == 'Number':
            continue
        
        keys = list(feature_counter[k].keys())
        if not keys:
            continue
        new_v_nat = random.choice(keys)
        if (k, new_v_nat) not in nat2pm:
            continue
        new_v_pm = nat2pm[(k, new_v_nat)]
        
        if bp.tag.aspect and bp.tag.aspect == 'perf' and new_v_pm == 'pres':
            continue
        
        if (k, new_v_nat) in nat2pm:
            new_feats.append(new_v_pm)
    random.shuffle(new_feats)
    infl = None
    newp = bp
    if new_feats:
        for f in new_feats:
            try:
                infl = newp.inflect({f})
            except ValueError:
                continue
            if infl:
                newp = infl
    if newp.word != bp.word:
        return newp.word
    return bp.word

In [ ]:
def spoil_text(doc, reinflection=0.5, important_drop=0.2, helper_drop=0.5, position_noise=2.0):
    results = []
    for token in doc.tokens:
        if token.pos in IMPORTANT_POS:
            if random.random() < important_drop:
                continue
        else:
            if random.random() < helper_drop:
                continue
        word = token.text
        if random.random() < reinflection:
            word = reinflect(token)
        results.append(word)
    if position_noise:
        orders = [random.normalvariate(mu=i, sigma=position_noise) for i, _ in enumerate(results)]
        results = [x for _, x in sorted(zip(orders, results))]
    if not results:
        return random.choice(doc.tokens).text.lower()
    return ' '.join(results).lower()

spoil_text(text2doc('Шла Саша по шоссе и сосала сушку.'), reinflection=0, important_drop=0, helper_drop=0)

'по шла саша и сосала сушку шоссе .'

In [ ]:
import numpy as np

def assemble_task():
    # text = random.choice(tapaco['train'])['paraphrase']
    text = random.choice(df_leipzig.text)
    spoiled = spoil_text(
        text2doc(text), 
        reinflection=random.random() * 1.0,
        important_drop=random.random() * 0.5,
        helper_drop=random.random() * 0.8, 
        position_noise=np.exp(random.normalvariate(0, 1)), 
    )
    return f'assemble | {spoiled}', text

assemble_task()

('assemble | раунде лучший можно любом по выбирается участвовать итогам результат',
 'Участвовать можно в любом раунде, по итогам выбирается лучший результат.')

### Question answering

Take the model from http://docs.deeppavlov.ai/en/master/features/models/squad.html

In [ ]:
!wget http://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz

--2021-06-10 21:03:06--  http://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz [following]
--2021-06-10 21:03:06--  https://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22825727 (22M) [application/octet-stream]
Saving to: ‘sber_squad-v1.1.tar.gz’

sber_squad-v1.1.tar 100%[===================>]  21.77M  13.9MB/s    in 1.6s    

2021-06-10 21:03:08 (13.9 MB/s) - ‘sber_squad-v1.1.tar.gz’ saved [22825727/22825727]



In [ ]:
!tar -xvzf sber_squad-v1.1.tar.gz

dev-v1.1.json
train-v1.1.json


In [ ]:
import json
with open('train-v1.1.json', 'r') as f:
    sbsq = json.load(f)

In [ ]:
len(sbsq['data'][0]['paragraphs'])

45328

In [ ]:
pp = random.choice(sbsq['data'][0]['paragraphs'])
pp

{'context': 'Новым главным тренером сборной Германии стал Эрих Риббек, ставший самым возрастным тренером сборной Германии за всю её историю (на момент начала работы ему исполнился 61 год). Кроме этого, Риббек запомнился ещё тем, что проработал со сборной меньше всех других главных тренеров (с 1998 по 2000 годы); исключил из сборной ветерана Лотара Маттеуса по причине его преклонного возраста, хотя и взял его после долгих уговоров на чемпионат Европы 2000 года и, самое главное — сборная под его руководством выступила хуже всего в своей истории, проиграв и Кубок конфедераций 1999 года, и Евро 2000. В обоих случаях команда даже не преодолела групповой этап.',
 'id': '3065',
 'qas': [{'answers': [{'answer_start': 167, 'text': '61 год'}],
   'id': '31533',
   'question': 'Сколько лет было Эриху Риббеку в начале работы главным тренером?'}]}

In [ ]:
def ask_task():
    pp = random.choice(sbsq['data'][0]['paragraphs'])
    qq = random.choice(pp['qas'])
    return f'ask | {pp["context"]}', qq["question"]

ask_task()

('ask | Наиболее спорной является койсанская гипотеза, согласно которой в одну макросемью объединяются все не-банту языки юга Африки, проживают в государствах: Намибия (62,1 %), Ботсвана (19,6 %), Танзания (13,4 %), Ангола (2,6 %), ЮАР (1 %), Зимбабве. Их общим признаком является наличие особых щёлкающих согласных. По этому же признаку к койсанским языкам добавляются два изолированных языка с востока Африки: сандаве и хадза. Койсанские языки изучены очень слабо, причем около половины из примерно 30 языков уже вымерло, а большинство остальных находится на грани вымирания. Все это значительно затрудняет их исследование. В середине 1980-х годов на африканском континенте насчитывалось 306 тыс. человек народов, принадлежащих к этой языковой макросемье, что составляло 0,06 % от всего населения Африки. Крупнейшими народами этой макросемьи являются готтентоты — 110 тыс. чел. (36 %), горные дамара — 80 (26 %), бушмены — 75 (24,5 %) и сандаве — 40 (13 %). Ранее по этнографическому принципу эти я

In [ ]:
def comprehend_task():
    pp = random.choice(sbsq['data'][0]['paragraphs'])
    qq = random.choice(pp['qas'])
    aa = random.choice(qq['answers'])
    return f'comprehend | {pp["context"]}.\nВопрос: {qq["question"]}', aa["text"]

comprehend_task()

('comprehend | В СССР теоретические и экспериментальные исследования особенностей пуска, работы и контроля реакторов были проведены группой физиков и инженеров под руководством академика И. В. Курчатова. Первый советский реактор Ф-1 был построен в Лаборатории № 2 АН СССР (Москва). Этот реактор выведен в критическое состояние 25 декабря 1946 года. Реактор Ф-1 был набран из графитовых блоков и имел форму шара диаметром примерно 7,5 м. В центральной части шара диаметром 6 м по отверстиям в графитовых блоках размещены урановые стержни. Реактор Ф-1, как и реактор CP-1, не имел системы охлаждения, поэтому работал на очень малых уровнях мощности. Результаты исследований на реакторе Ф-1 стали основой проектов более сложных по конструкции промышленных реакторов. В 1948 году введён в действие реактор И-1 (по другим данным он назывался А-1) по производству плутония, а 27 июня 1954 года вступила в строй первая в мире атомная электростанция электрической мощностью 5 МВт в г. Обнинске..\nВопрос: Под

In [ ]:
pd.Series([len(tokenizer.tokenize(random.choice(sbsq['data'][0]['paragraphs'])['context'])) for _ in range(1000)]).quantile([0.5, 0.75, 0.9, 0.95, 0.99, 1])

0.50     207.00
0.75     261.00
0.90     326.00
0.95     361.00
0.99     558.25
1.00    1366.00
dtype: float64

In [ ]:
pd.Series([len(tokenizer.tokenize(comprehend_task()[0])) for _ in range(3000)]).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

0.25     198.00
0.50     232.00
0.75     288.00
0.90     355.00
0.95     390.00
0.99     500.02
1.00    1039.00
dtype: float64

### Russian SuperGLUE

In [ ]:
!for TASK in LiDiRus RCB PARus MuSeRC TERRa RUSSE RWSD DaNetQA RuCoS; do wget https://russiansuperglue.com/tasks/download/$TASK --content-disposition && unzip $TASK.zip; done
!rm -rf ./*.zip

--2021-06-10 21:03:11--  https://russiansuperglue.com/tasks/download/LiDiRus
Resolving russiansuperglue.com (russiansuperglue.com)... 37.18.107.48
Connecting to russiansuperglue.com (russiansuperglue.com)|37.18.107.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47118 (46K) [application/zip]
Saving to: ‘LiDiRus.zip’

LiDiRus.zip         100%[===================>]  46.01K  --.-KB/s    in 0.04s   

2021-06-10 21:03:12 (1.12 MB/s) - ‘LiDiRus.zip’ saved [47118/47118]

Archive:  LiDiRus.zip
   creating: LiDiRus/
  inflating: LiDiRus/.DS_Store       
   creating: __MACOSX/
   creating: __MACOSX/LiDiRus/
  inflating: __MACOSX/LiDiRus/._.DS_Store  
  inflating: LiDiRus/LiDiRus.jsonl   
  inflating: __MACOSX/LiDiRus/._LiDiRus.jsonl  
  inflating: __MACOSX/._LiDiRus      
--2021-06-10 21:03:12--  https://russiansuperglue.com/tasks/download/RCB
Resolving russiansuperglue.com (russiansuperglue.com)... 37.18.107.48
Connecting to russiansuperglue.com (russiansuperglue.co

In [ ]:
import codecs
import json
import pandas as pd
from collections import defaultdict
import copy
import random
import numpy as np
import re

def load_jsonl(path):
    with codecs.open(path, encoding='utf-8-sig') as reader:
        lines = reader.read().split("\n")
        lines = list(map(json.loads, filter(None, lines)))
    return lines

In [ ]:
TASK_NAMES = ['LiDiRus', 'RCB', 'PARus', 'MuSeRC', 'TERRa', 'RUSSE', 'RWSD', 'DaNetQA', 'RuCoS']

task_data = {k: load_jsonl(f'{k}/train.jsonl') for k in TASK_NAMES[1:]}

task_ids = list(task_data.keys())
task_weights = np.array([len(task_data[t]) for t in task_ids])**0.5
task_weights /= task_weights.sum()
print(task_ids)
print(task_weights)

['RCB', 'PARus', 'MuSeRC', 'TERRa', 'RUSSE', 'RWSD', 'DaNetQA', 'RuCoS']
[0.03544588 0.03387339 0.0378716  0.08662586 0.23859111 0.04169318
 0.07083104 0.45506794]


In [ ]:
RCB_labels = {
    'contradiction': 'противоречие',
    'entailment': 'следствие',
    'neutral': 'не очевидно'
}

def RCB_sampler(item):
    return f'Дано: {item["premise"]}. Гипотеза: {item["hypothesis"]}. Логично?', RCB_labels[item["label"]]

def PARus_sampler(item):
    if item['question'] == 'cause':
        q, a = 'Следствие', 'Причина'
    else: # effect
        q, a = 'Причина', 'Следствие'
    idx = random.choice([0, 1])
    ch = item['choice' + str(idx + 1)]
    ans = 'да' if item['label'] == idx else 'нет'
    parts = [f'{q}: {item["premise"]}', f"{a}: {ch}"]
    random.shuffle(parts)
    return f'{parts[0]}. {parts[1]}. Логично?', ans

def MuSeRC_sampler(item):
    text = re.sub('\(\d+\) ', '', item['passage']['text'])
    qq = random.choice(item['passage']['questions'])
    q = qq['question']
    a = random.choice([a for a in qq['answers'] if a['label']])['text']
    return f'Вопрос: {q}. Дано: {text}', a

TERRa_labels = {
    'entailment': 'логично',
    'not_entailment': 'не очевидно'
}

def TERRa_sampler(item):
    return f'Дано: {item["premise"]}. Гипотеза: {item["hypothesis"]}. Логично?', TERRa_labels[item["label"]]

def RUSSE_sampler(item):
    sents = [item['sentence1'], item['sentence2']]
    random.shuffle(sents)
    ans = 'да' if item['label'] else 'нет'
    return f'Слово "{item["word"]}" употребляется в одинаковом смысле? Текст 1: {sents[0]}. Текст 2: {sents[1]}.', ans

def RWSD_sampler(item):
    s1, s2 = item['target']['span1_text'], item['target']['span2_text']
    if item['label'] and random.random() < 0.5:
        return f'{item["text"]}. К чему относится "{s2}"?', s1
    return f'{item["text"]}. "{s2}" относится к "{s1}"?', 'да' if item['label'] else 'нет'

def DaNetQA_sampler(item):
    text = item["passage"]
    q = item["question"].replace('?', '')
    ans = 'да' if item['label'] else 'нет'
    if random.random() < 0.5:
        return f'{text}. Вопрос: {q}?', ans
    else:
        return f'Вопрос: {q}? {text}', ans

def RuCoS_sampler(item):
    text = item["passage"]['text'].split('@highlight')[0].strip()
    qa = random.choice(item['qas'])
    q, a = qa['query'], random.choice(qa['answers'])['text']
    if random.random() < 0.5:  #summarization
        return text + '\nВкратце:', q.replace('@placeholder', a)
    else:
        summary = q.replace('@placeholder', 'ЭТО')
        return f'{summary}. {text}. Вопрос: ЭТО - что?', a

samplers = {
    'RCB': RCB_sampler,
    'PARus': PARus_sampler,
    'MuSeRC': MuSeRC_sampler,
    'TERRa': TERRa_sampler,
    'RUSSE': RUSSE_sampler,
    'RWSD': RWSD_sampler,
    'DaNetQA': DaNetQA_sampler,
    'RuCoS': RuCoS_sampler,
}

def rsg_task(task=None):
    if task is None:
        task = random.choices(task_ids, weights=task_weights)[0]
    q, a = samplers[task](random.choice(task_data[task]))
    return f'{task} | {q}', a

In [ ]:
for t in task_ids:
    q, a = rsg_task(task=t)
    print(q)
    print(a)
    print()

RCB | Дано: Квартира ему не понравилась, он мне долго объяснял, почему — как я понял, вид из окна был не тот, мешал ему работать. Получил квартиру рядом, в другом таком же доме — кстати, при входе в этот столь респектабельный дом на него, стараясь отобрать сумку, напал грабитель и нанес ему несколько ран. Началась эпопея ремонта и приведения квартиры в тот образцовый порядок, который он считал необходимым, единственно возможным.. Гипотеза: Образцовый порядок необходим, единственно возможен.. Логично?
не очевидно

PARus | Причина: Мальчик боролся со своим старшим братом.. Следствие: Мальчик подражал старшему брату.. Логично?
нет

MuSeRC | Вопрос: Почему Батлер хочет убить Фрэнка?. Дано: Чикагский пожар 7 октября 1871 года в момент разрушает финансовое благополучие Каупервуда. Огонь, охвативший торговую часть города, вызывает биржевую панику. Фрэнк пустил в оборот пятьсот тысяч из городской казны, и теперь это самая большая его проблема. Стинера нет в городе, и Каупервуд решает начистоту

### Train the model

In [ ]:
# raw_model = '/gd/MyDrive/models/rut5-base-raw'  # start fine-tuning
raw_model = '/gd/MyDrive/models/rut5-base-partial'  # continue fine-tuning
model = T5ForConditionalGeneration.from_pretrained(raw_model)
tokenizer = T5Tokenizer.from_pretrained(raw_model)

In [ ]:
device = torch.device('cuda')
model.to(device);

In [ ]:
optimizer = torch.optim.Adam(params = [p for p in model.parameters() if p.requires_grad], lr=1e-5)

```
translate_task       1.56
paraphrase_task      1.55
fill_gap_task        3.21
assemble_task        1.51
simplify_task        1.10
reply_task           3.27
answer_task          3.91
ask_task             1.38
comprehend_task      0.38
headline_task        1.71
quiz_task            4.94
```

In [ ]:
TASKS = [
    quiz_task,
    answer_task,
    reply_task,
    fill_gap_task,
    assemble_task,
    translate_task,
    headline_task,
    paraphrase_task,
    ask_task, 
    rsg_task,
    simplify_task, 
    comprehend_task,
]
# omit sumarize_task because texts are too long
len(TASKS)

12

In [ ]:
def predict(x, n=3):
    inputs = tokenizer(x, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        hypotheses = model.generate(
            **inputs, 
            do_sample=True, 
            top_p=0.9, 
            num_return_sequences=3, 
            repetition_penalty=2.5,
            max_length=64,
        )
    return [tokenizer.decode(h, skip_special_tokens=True) for h in hypotheses]

In [ ]:
model.eval()

for t in TASKS:
    x, y = t()
    print(x, ' \n --> ', y)
    for p in predict(x, n=3):
        print(p)
    print()

quiz | Спортивная встреча для установления первенства по виду спорта  
 -->  состязание
воркаут
на льду
атлетика

answer | Какие празднечные конкурсы или игры вы знаете?  
 -->  вопрос-ответ, на одной бумажке пишется вопрос, на другой - ответ, бумажек лучше много, вопросы кладутся в одну шапку, ответы - в другую. Гости по-очереди вытаскивают сначала вопрос, говорит, кому отвечать и другой вытаскивает ответ, получается смешно! К примеру, вопрос: "Вы свою жену любите? " а ответ на это попался: "Только в большой компании! " - это у нас так было. Вот тебе примерные вопросы и ответы: Игра «Вопрос, ответ» . <br> <br>Вариант 1 <br>Вопросы: <br> <br>1.Хотели бы вы стать миллионером? <br>2.Вам хотелось бы попасть в гарем? <br>3.Вы часто бываете в ресторане? <br>4.Вы с удовольствием ходите на работу? <br>5.Могли бы вы родить богатыря? <br>6.Носите ли вы парик? <br>7.У вас есть недостатки? <br>8.Вы любите музыку? <br>9.Вы всегда так вежливы как сегодня? <br>10.Способны ли вы любить? <br>11.Вы с у

In [ ]:
for t in task_ids:
    q, a = rsg_task(task=t)
    print(q)
    print(a)
    for p in predict(q, n=3):
        print(p)
    print()

RCB | Дано: И тут на меня нашло. Как помню, в первый раз жизни защемило сердце. Позже стал осознавать, что так проявляет себя моя тревожность.. Гипотеза: Так проявляет себя моя тревожность.. Логично?
следствие
следствие
не очевидно
следствие

PARus | Следствие: Пожарная сигнализация сработала.. Причина: Я зажёг свечу.. Логично?
нет
нет
нет
да

MuSeRC | Вопрос: Почему Эмпедокл терзается и чувствует себя бессильным и опустошённым?. Дано: Мужчины со злорадством рассуждают: сдал Эмпедокл, и поделом ему. Слишком много возомнил о себе, открыл черни божественные тайны, которым надлежало оставаться достоянием одних жрецов. Вредным было его влияние на народ — все эти дерзкие речи о новой жизни, которая должна заменить старый, привычный быт, призывы не покоряться исконным обычаям и традиционным верованиям. Человек не должен нарушать положенные ему пределы, бунтарство обернулось для Эмпедокла поражением. Поскольку он удалился от всех, прошла молва, что боги взяли его живым на небо. Народ привык с

In [ ]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [ ]:
optimizer.param_groups[0]['lr'] = 1e-5

The easiest tasks seem to be `simplify` and `comprehend` (because the correct response is obvious from the text and is short), the most difficult are `translate` (probably because languages are not aligned),  `fill_gap` (surprisingly, because T5 was trained to do it), `reply` and `answer` (okay, here the answer is unpredictable). 

In [ ]:
def eval_losses(n=10, max_len=1024):
    for task in TASKS:
        tot = 0
        for i in range(n):
            xxx, yyy = task()
            x = tokenizer(xxx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
            y = tokenizer(yyy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)

            loss = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss
            loss.backward()
            tot += loss.item()
        print(f'{task.__name__:20s} {tot / n :2.2f}')

eval_losses(n=20)

quiz_task            3.54
answer_task          3.54
reply_task           2.99
fill_gap_task        2.53
assemble_task        1.57
translate_task       2.15
headline_task        1.55
paraphrase_task      0.80
ask_task             0.80
rsg_task             0.93
simplify_task        0.88
comprehend_task      0.26


One round of fine-tuning lasts until Colab stops
* May 11: loss on 10 tasks goes from 1.89 to 1.69 roughly
* May 12: loss on 10 tasks stays about 1.69 for 15K batches 
* May 13: loss still flat about 1.69-1.67. Change acc.steps from 8 to 32, then after time loss goes to 1.62-1.64
* May 27: use a larger GPU (colab pro), batch 4 instead of 2. Loss is 1.27-1.24 because of more `<pad>` tokens -> after a night, 1.21-1.22
* Add the quiz task, loss goes back up to 1.22-12.4, but decreases to 1.19 after some training
* reduce batch to 2, loss goes back to 1.63 (but now with a more difficult quiz task)
* May 28: add russian SuperGlue, loss goes up to 1.73, then down to 1.50 - 1.55

In [2]:
from tqdm import tqdm_notebook

In [ ]:
model.train();
batch_size = 2
max_len = 1024
epochs = 5
accumulation_steps = 32
save_steps = 5000

window = 5000
ewm = 0

tq = trange(int(100000000 / batch_size))
cleanup()

for i in tq:
    xx = []
    yy = []
    for _ in range(batch_size):
        xxx, yyy = random.choice(TASKS + [rsg_task] * 3)()  # rsg is more various, increase its occurrence 4-fold
        xx.append(xxx)
        yy.append(yyy)

    try:
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        # do not force the model to predict pad tokens
        y.input_ids[y.input_ids==0] = -100

        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        # print('ok')
    except RuntimeError:
        print([xxx.split(' |')[0] for xxx in xx])
        loss = None
        cleanup()
        continue

    w = 1 / min(i+1, window)
    ewm = ewm * (1-w) + loss.item() * w
    tq.set_description(f'loss: {ewm}')
    
    if i % accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        cleanup()
    
    if i % window == 0 and i > 0:
        print(ewm)
        cleanup()
        # optimizer.param_groups[0]['lr'] *= 0.999
    if i % save_steps == 0 and i > 0:
        model.save_pretrained(MODEL_NAME)
        tokenizer.save_pretrained(MODEL_NAME)
        print('saving...', i, optimizer.param_groups[0]['lr'])

        try:
            optimizer.step()
            optimizer.zero_grad()
            eval_losses()
            optimizer.step()
            optimizer.zero_grad()
        except RuntimeError:
            cleanup()

['RUSSE', 'translate en-ru']
['translate en-ru', 'RuCoS']
1.5169176257705035
saving... 5000 1e-05
quiz_task            3.92
answer_task          3.97
reply_task           3.50
fill_gap_task        3.29
assemble_task        2.47
translate_task       1.94
headline_task        2.13
paraphrase_task      1.58
ask_task             1.84
rsg_task             1.37
simplify_task        0.64
comprehend_task      0.39
1.5029236435164788
saving... 10000 1e-05
quiz_task            4.45
answer_task          3.96
reply_task           3.64
fill_gap_task        3.94
assemble_task        2.74
translate_task       2.51
headline_task        2.03
paraphrase_task      1.50
ask_task             1.35
rsg_task             1.62
simplify_task        0.89
comprehend_task      0.76
['paraphrase', 'translate ru-en']
1.5008431054809206
saving... 15000 1e-05
quiz_task            4.35
answer_task          4.38
reply_task           3.12
fill_gap_task        3.98
assemble_task        2.12
translate_task       1.32
headli

ValueError: ignored

In [ ]:
x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
y.input_ids[y.input_ids==0] = -100

loss = model(
    input_ids=x.input_ids,
    attention_mask=x.attention_mask,
    labels=y.input_ids,
    decoder_attention_mask=y.attention_mask,
    return_dict=True
).loss
loss.backward()

In [ ]:
model.save_pretrained(MODEL_NAME)
tokenizer.save_pretrained(MODEL_NAME)

In [ ]:
!ls $MODEL_NAME